# Predict Survival of the Titanic Passengers

Dataset Source: [https://ww2.amstat.org/publications/jse/v3n3/datasets.dawson.html](https://ww2.amstat.org/publications/jse/v3n3/datasets.dawson.html)

## Table of Contents
- [Load Libraries](#load_libraries)
- [Access data](#access_data)
- [Split Data into Training and Test set](#training_test)
- [Build Logistic Regression Model](#build_model)
- [Predict for Test data](#test_data)
- [Evaluate the Model](#evaluate_model)

<a id="load_libraries"></a>
## Load Libraries

The Spark and Python libraries that you need are preinstalled in the notebook environment and only need to be loaded.

Run the following cell to load the libraries you will work with in this notebook:

In [1]:
# PySpark Machine Learning Library
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row, SQLContext

import os
import sys
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from numpy import array

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Library for confusion matrix, precision, test error
from pyspark.mllib.evaluation import MulticlassMetrics
# Library For Area under ROC curve and Area under precision-recall curve
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Assign resources to the application
sqlContext = SQLContext(sc)

In [2]:
# The data will be loaded into an array.
# This is the summary of the data structure, including the column position and name.
# The first filed starts from position 0. 

# 0 Name    -  Passenger first and last name.
# 1 PClass  -  Passenger class (1st, 2nd, or 3rd)
# 2 Age
# 3 Sex
# 4 Survived -  1 if the passenger survived;  0 if the passenger did not survive
# 5 PersonID

# Label is a target variable. PersonInfo is a list of independent variables besides unique identifier

LabeledDocument = Row("PersonID", "PersonInfo", "label")

# Define a function that parses the raw CSV file and returns an object of type LabeledDocument

def parseDocument(line):
    values = [str(x) for x in line.split(',')] 
    if (values[4]>'0'):
      Survived = 1.0
    else:
      Survived = 0.0
        
    textValue = str(values[1]) + " " + str(values[2])+" " + str(values[3])
    return LabeledDocument(values[5], textValue, Survived)

<a id="access_data"></a>
## Access Data
Before you can access data in the data file in the Object Storage, you must setup the Spark configuration with your Object Storage credentials. 

To do this, click on the cell below and select the **Insert to code > Insert Spark Session DataFrame** function from the Files tab below the data file you want to work with.

<div class="alert alert-block alert-info">The following code contains the credentials for a file in your IBM Cloud Object Storage. Delete the code starting from `from pyspark.sql import SparkSession` line before you run the cell.</div>

In [3]:
# Object Storage Credentials
import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'lue6np0RwcvhARfJIQNtvVTUt3I45m5qk9UHM6LFTc3B',
    'service_id': 'iam-ServiceId-3ac1bdb0-c8a1-4ae8-abe0-2dd53ae4c6e9',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_9d550c7f6655453f915511631f5b077f_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

Now let's load the data into a `Spark RDD` and output the number of rows and first 5 rows.
Each project you create has a bucket in your object storage. You can get the bucket name from the project Settings page. Change the string `BUCKET` to the bucket name

In [4]:
data = sc.textFile(cos.url('Titanic.csv', 'assignment331c5c83422a7434ab113135bf62f5fb0'))
print "Total records in the data set:", data.count()
print "The first 5 rows"
data.take(5)

Total records in the data set: 757
The first 5 rows


[u'Name,PClass,Age,Sex,Survived,PersonID',
 u'Allen Miss Elisabeth Walton,1st,29,female,1,1',
 u'Allison Miss Helen Loraine,1st,2,female,0,2',
 u'Allison Mr Hudson Joshua Creighton,1st,30,male,0,3',
 u'Allison Mrs Hudson JC (Bessie Waldo Daniels),1st,25,female,0,4']

Crate DataFrame from RDD

In [5]:
#Load the data into a dataframe, parse it using the function above
documents = data.filter(lambda s: "Name" not in s).map(parseDocument)
TitanicData = documents.toDF() # ToDataFrame
print "Number of records: " + str(TitanicData.count())
print "First 5 records: "
TitanicData.take(5)

Number of records: 756
First 5 records: 


[Row(PersonID=u'1', PersonInfo=u'1st 29 female', label=1.0),
 Row(PersonID=u'2', PersonInfo=u'1st 2 female', label=0.0),
 Row(PersonID=u'3', PersonInfo=u'1st 30 male', label=0.0),
 Row(PersonID=u'4', PersonInfo=u'1st 25 female', label=0.0),
 Row(PersonID=u'5', PersonInfo=u'1st 0.92 male', label=1.0)]

<a id="training_test"></a>
## Split Data into Training and Test set

We divide the data into training and test set.  The training set is used to build the model to be used on future data, and the test set is used to evaluate the model.

In [6]:
# Divide the data into training and test set
(train, test) = TitanicData.randomSplit([0.8, 0.2])
print "Number of records in the training set: " + str(train.count())
print "Number of records in the test set: " + str(test.count())
# Output first 20 records in the training set
print "First 20 records in the training set: "
train.show()

Number of records in the training set: 624
Number of records in the test set: 132
First 20 records in the training set: 
+--------+-------------+-----+
|PersonID|   PersonInfo|label|
+--------+-------------+-----+
|       1|1st 29 female|  1.0|
|      10|  1st 71 male|  0.0|
|     100|  1st 46 male|  0.0|
|     101|  1st 25 male|  1.0|
|     102|1st 21 female|  1.0|
|     104|1st 49 female|  1.0|
|     106|1st 36 female|  1.0|
|     107|  1st 55 male|  0.0|
|     108|1st 52 female|  1.0|
|     109|1st 24 female|  1.0|
|      11|  1st 47 male|  0.0|
|     110|1st 16 female|  1.0|
|     112|1st 51 female|  1.0|
|     113|  1st 42 male|  0.0|
|     114|1st 35 female|  1.0|
|     115|  1st 35 male|  1.0|
|     116|  1st 38 male|  1.0|
|     117|1st 35 female|  1.0|
|     118|1st 50 female|  0.0|
|     119|  1st 49 male|  1.0|
+--------+-------------+-----+
only showing top 20 rows



<a id="build_model"></a>
## Build Logistic Regression Model

We use the Pipeline of SparkML to build the Logistic Regression Model

In [7]:
# set up Logistic Regression using Pipeline of SparkML
tokenizer = Tokenizer(inputCol="PersonInfo", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [8]:
# set up Logistic Regression Model
# the stages are executed in order
model = pipeline.fit(train)
#[stage.coefficients for stage in model.stages if hasattr(stage, "coefficients")]
# model.stages[2].intercept
model.stages[2].coefficients

SparseVector(262144, {1251: -3.0553, 4106: -3.18, 8612: -1.0636, 18659: 0.1234, 23689: -0.1167, 24537: -0.236, 31351: 0.1305, 32672: -0.322, 37812: 0.359, 43733: -1.3179, 56811: 0.3897, 60486: 1.0426, 63384: -1.0556, 64358: 0.2618, 68516: -1.0626, 69821: -0.3466, 78326: -1.5195, 82742: 3.5571, 83214: 0.0855, 83863: -0.3072, 89074: 4.8292, 89689: 1.5756, 98627: -0.9334, 98962: 3.7439, 102033: -1.3534, 104268: -2.9561, 109681: -1.3979, 110466: -0.0096, 114415: 0.085, 122111: 1.9757, 122295: 1.3757, 127663: -0.7429, 128319: 0.4579, 130010: 0.4102, 130972: -1.5104, 134360: -0.9008, 139093: 1.5487, 146429: 1.1346, 147946: -0.2807, 153779: -0.8924, 155743: 3.2298, 162286: 0.047, 167741: -3.4939, 173170: 0.3013, 175329: 0.4599, 177493: 0.8597, 177523: -1.2967, 178347: -0.574, 180089: 4.5844, 186753: -1.7762, 187043: 0.0149, 191455: -0.2434, 197774: 4.5844, 199158: -3.1773, 201434: 1.4816, 206255: 1.3514, 207020: 0.1765, 212053: -0.1292, 213217: 0.5011, 219381: 0.2654, 230679: -0.6161, 232155:

<a id="test_data"></a>
## Predict for Test data

In [9]:
# Make predictions on test documents and print columns of interest
prediction = model.transform(test)
selected = prediction.select("PersonInfo", "prediction", "probability")
for row in selected.collect():
    print row
#for row in prediction.collect():
#    print row

Row(PersonInfo=u'1st 48 male', prediction=0.0, probability=DenseVector([0.581, 0.419]))
Row(PersonInfo=u'1st 45 male', prediction=0.0, probability=DenseVector([0.5084, 0.4916]))
Row(PersonInfo=u'1st 44 female', prediction=1.0, probability=DenseVector([0.1559, 0.8441]))
Row(PersonInfo=u'1st 41 male', prediction=0.0, probability=DenseVector([0.7217, 0.2783]))
Row(PersonInfo=u'1st 30 male', prediction=0.0, probability=DenseVector([0.7583, 0.2417]))
Row(PersonInfo=u'1st 64 male', prediction=0.0, probability=DenseVector([0.8793, 0.1207]))
Row(PersonInfo=u'1st 46 male', prediction=0.0, probability=DenseVector([0.9632, 0.0368]))
Row(PersonInfo=u'1st 33 female', prediction=1.0, probability=DenseVector([0.0627, 0.9373]))
Row(PersonInfo=u'1st 13 male', prediction=0.0, probability=DenseVector([0.8483, 0.1517]))
Row(PersonInfo=u'1st 50 male', prediction=0.0, probability=DenseVector([0.7251, 0.2749]))
Row(PersonInfo=u'1st 24 male', prediction=0.0, probability=DenseVector([0.5486, 0.4514]))
Row(Pers

In [10]:
#Tabulate the predicted outcome
prediction.select("prediction").groupBy("prediction").count().show(truncate=False)

+----------+-----+
|prediction|count|
+----------+-----+
|0.0       |90   |
|1.0       |42   |
+----------+-----+



In [11]:
#Tabulate the actual outcome
prediction.select("label").groupBy("label").count().show(truncate=False)

+-----+-----+
|label|count|
+-----+-----+
|0.0  |80   |
|1.0  |52   |
+-----+-----+



In [12]:
# This table shows:
# 1. The number of survived passengers predicted as survived
# 2. The number of survived passengers predicted as not survived
# 3. The number of not survived passengers predicted as survived
# 4. The number of not survived passengers predicted as not survived

prediction.crosstab('label', 'prediction').show()

+----------------+---+---+
|label_prediction|0.0|1.0|
+----------------+---+---+
|             1.0| 21| 31|
|             0.0| 69| 11|
+----------------+---+---+



<a id="evaluate_model"></a>
## Evaluate the Model

We evaluate the model on a training set and on a test set.  The purpose is to measure the model's predictive accuracy, including the accuracy for new data.

In [13]:
# Evaluate the Logistic Regression model on a training set
# Select (prediction, true label) and compute training error
pred_lr=model.transform(train).select("prediction", "label")
eval_lr=MulticlassClassificationEvaluator (
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy_lr=eval_lr.evaluate(pred_lr)
# create RDD
predictionAndLabels_lr=pred_lr.rdd
metrics_lr=MulticlassMetrics(predictionAndLabels_lr)
precision_lr=metrics_lr.precision(1.0)
recall_lr=metrics_lr.recall(1.0)
f1Measure_lr = metrics_lr.fMeasure(1.0, 1.0)
print("F1 Measure = %s" % f1Measure_lr)
print ("Training Accuracy = %s" %accuracy_lr)
print ("Training Error = %s" % (1-accuracy_lr))
print ("Precision = %s" %precision_lr)
print ("Recall = %s" %recall_lr)

F1 Measure = 0.802348336595
Training Accuracy = 0.838141025641
Training Error = 0.161858974359
Precision = 0.82
Recall = 0.785440613027


In [14]:
# Evaluate the Logistic Regression model on a test set
# Select (prediction, true label) and compute test error
pred_lr=model.transform(test).select("prediction", "label")
eval_lr=MulticlassClassificationEvaluator (
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy_lr=eval_lr.evaluate(pred_lr)
# create RDD
predictionAndLabels_lr=pred_lr.rdd
metrics_lr=MulticlassMetrics(predictionAndLabels_lr)
precision_lr=metrics_lr.precision(1.0)
recall_lr=metrics_lr.recall(1.0)
f1Measure_lr = metrics_lr.fMeasure(1.0, 1.0)
print("F1 Measure = %s" % f1Measure_lr)
print ("Test Accuracy = %s" %accuracy_lr)
print ("Test Error = %s" % (1-accuracy_lr))
print ("Precision = %s" %precision_lr)
print ("Recall = %s" %recall_lr)

F1 Measure = 0.659574468085
Test Accuracy = 0.757575757576
Test Error = 0.242424242424
Precision = 0.738095238095
Recall = 0.596153846154


In [15]:
bin_lr=BinaryClassificationMetrics(predictionAndLabels_lr)

# Area under precision-recall curve
print("Area under PR = %s" % bin_lr.areaUnderPR)
# Area under precision-recall curve
print("Area under ROC = %s" % bin_lr.areaUnderROC)

Area under PR = 0.74666999667
Area under ROC = 0.729326923077
